In [1]:
import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import WebKB
from torch_geometric.loader import DataLoader

In [2]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[251, 1703], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515])


In [3]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

1200
800
510


# GSAGE

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [5]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
idx=[data.train_mask[i][0] for i in range(183)]
train_index = np.where(idx)[0]
print(train_index)

In [6]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8861, Train: 66.67%, Valid: 45.00% Test: 33.33%
Run: 01, Epoch: 02, Loss: 0.9793, Train: 88.33%, Valid: 62.50% Test: 60.78%
Run: 01, Epoch: 03, Loss: 0.7278, Train: 90.00%, Valid: 68.75% Test: 66.67%
Run: 01, Epoch: 04, Loss: 0.6498, Train: 90.83%, Valid: 71.25% Test: 62.75%
Run: 01, Epoch: 05, Loss: 0.5245, Train: 90.00%, Valid: 71.25% Test: 64.71%
Run: 01, Epoch: 06, Loss: 0.5035, Train: 90.00%, Valid: 70.00% Test: 66.67%
Run: 01, Epoch: 07, Loss: 0.4193, Train: 89.17%, Valid: 71.25% Test: 68.63%
Run: 01, Epoch: 08, Loss: 0.4050, Train: 92.50%, Valid: 71.25% Test: 68.63%
Run: 01, Epoch: 09, Loss: 0.4147, Train: 93.33%, Valid: 71.25% Test: 68.63%
Run: 01, Epoch: 10, Loss: 0.4192, Train: 93.33%, Valid: 71.25% Test: 68.63%
Run: 01, Epoch: 11, Loss: 0.3270, Train: 93.33%, Valid: 71.25% Test: 68.63%
Run: 01, Epoch: 12, Loss: 0.2525, Train: 94.17%, Valid: 71.25% Test: 68.63%
Run: 01, Epoch: 13, Loss: 0.2610, Train: 94.17%, Valid: 71.25% Test: 72.55%
Run: 01, Epo

Run: 02, Epoch: 36, Loss: 0.0678, Train: 100.00%, Valid: 73.75% Test: 80.39%
Run: 02, Epoch: 37, Loss: 0.0561, Train: 100.00%, Valid: 73.75% Test: 80.39%
Run: 02, Epoch: 38, Loss: 0.0380, Train: 100.00%, Valid: 73.75% Test: 80.39%
Run: 02, Epoch: 39, Loss: 0.0655, Train: 100.00%, Valid: 73.75% Test: 80.39%
Run: 02, Epoch: 40, Loss: 0.0550, Train: 100.00%, Valid: 73.75% Test: 80.39%
Run: 02, Epoch: 41, Loss: 0.0436, Train: 100.00%, Valid: 73.75% Test: 82.35%
Run: 02, Epoch: 42, Loss: 0.0733, Train: 100.00%, Valid: 73.75% Test: 82.35%
Run: 02, Epoch: 43, Loss: 0.0677, Train: 100.00%, Valid: 73.75% Test: 82.35%
Run: 02, Epoch: 44, Loss: 0.0495, Train: 99.17%, Valid: 73.75% Test: 82.35%
Run: 02, Epoch: 45, Loss: 0.0456, Train: 100.00%, Valid: 75.00% Test: 82.35%
Run: 02, Epoch: 46, Loss: 0.0295, Train: 100.00%, Valid: 75.00% Test: 82.35%
Run: 02, Epoch: 47, Loss: 0.0300, Train: 100.00%, Valid: 75.00% Test: 82.35%
Run: 02, Epoch: 48, Loss: 0.0326, Train: 100.00%, Valid: 75.00% Test: 82.35%


Run: 03, Epoch: 44, Loss: 0.0525, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 03, Epoch: 45, Loss: 0.0395, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 03, Epoch: 46, Loss: 0.0311, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 03, Epoch: 47, Loss: 0.0688, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 03, Epoch: 48, Loss: 0.0347, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 03, Epoch: 49, Loss: 0.0422, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 03, Epoch: 50, Loss: 0.0894, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 03, Epoch: 51, Loss: 0.0370, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 03, Epoch: 52, Loss: 0.0637, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 03, Epoch: 53, Loss: 0.0504, Train: 100.00%, Valid: 70.00% Test: 62.75%
Run: 03, Epoch: 54, Loss: 0.0397, Train: 100.00%, Valid: 70.00% Test: 62.75%
Run: 03, Epoch: 55, Loss: 0.0417, Train: 100.00%, Valid: 71.25% Test: 62.75%
Run: 03, Epoch: 56, Loss: 0.0339, Train: 100.00%, Valid: 71.25% Test: 62.75%

Run: 04, Epoch: 85, Loss: 0.0135, Train: 100.00%, Valid: 72.50% Test: 76.47%
Run: 04, Epoch: 86, Loss: 0.0196, Train: 100.00%, Valid: 72.50% Test: 76.47%
Run: 04, Epoch: 87, Loss: 0.0196, Train: 100.00%, Valid: 72.50% Test: 76.47%
Run: 04, Epoch: 88, Loss: 0.0222, Train: 100.00%, Valid: 72.50% Test: 76.47%
Run: 04, Epoch: 89, Loss: 0.0249, Train: 100.00%, Valid: 72.50% Test: 78.43%
Run: 04, Epoch: 90, Loss: 0.0280, Train: 100.00%, Valid: 72.50% Test: 78.43%
Run: 04, Epoch: 91, Loss: 0.0093, Train: 100.00%, Valid: 72.50% Test: 78.43%
Run: 04, Epoch: 92, Loss: 0.0217, Train: 100.00%, Valid: 71.25% Test: 78.43%
Run: 04, Epoch: 93, Loss: 0.0075, Train: 100.00%, Valid: 71.25% Test: 78.43%
Run: 04, Epoch: 94, Loss: 0.0240, Train: 100.00%, Valid: 71.25% Test: 78.43%
Run: 04, Epoch: 95, Loss: 0.0117, Train: 100.00%, Valid: 71.25% Test: 78.43%
Run: 04, Epoch: 96, Loss: 0.0194, Train: 100.00%, Valid: 71.25% Test: 78.43%
Run: 04, Epoch: 97, Loss: 0.0173, Train: 100.00%, Valid: 71.25% Test: 78.43%

Run: 05, Epoch: 94, Loss: 0.0057, Train: 100.00%, Valid: 71.25% Test: 74.51%
Run: 05, Epoch: 95, Loss: 0.0268, Train: 100.00%, Valid: 71.25% Test: 72.55%
Run: 05, Epoch: 96, Loss: 0.0119, Train: 100.00%, Valid: 71.25% Test: 72.55%
Run: 05, Epoch: 97, Loss: 0.0071, Train: 100.00%, Valid: 71.25% Test: 74.51%
Run: 05, Epoch: 98, Loss: 0.0148, Train: 100.00%, Valid: 71.25% Test: 74.51%
Run: 05, Epoch: 99, Loss: 0.0100, Train: 100.00%, Valid: 71.25% Test: 74.51%
Run: 05, Epoch: 100, Loss: 0.0193, Train: 100.00%, Valid: 71.25% Test: 72.55%
Run 05:
Highest Train: 100.00
Highest Valid: 73.75
  Final Train: 90.00
   Final Test: 66.67
Run: 06, Epoch: 01, Loss: 1.5982, Train: 77.50%, Valid: 50.00% Test: 49.02%
Run: 06, Epoch: 02, Loss: 0.7250, Train: 80.83%, Valid: 62.50% Test: 58.82%
Run: 06, Epoch: 03, Loss: 0.5967, Train: 86.67%, Valid: 62.50% Test: 62.75%
Run: 06, Epoch: 04, Loss: 0.5208, Train: 88.33%, Valid: 65.00% Test: 60.78%
Run: 06, Epoch: 05, Loss: 0.4653, Train: 88.33%, Valid: 66.25% 

Run: 07, Epoch: 28, Loss: 0.1086, Train: 99.17%, Valid: 72.50% Test: 76.47%
Run: 07, Epoch: 29, Loss: 0.1020, Train: 99.17%, Valid: 72.50% Test: 76.47%
Run: 07, Epoch: 30, Loss: 0.1077, Train: 99.17%, Valid: 71.25% Test: 76.47%
Run: 07, Epoch: 31, Loss: 0.0923, Train: 99.17%, Valid: 70.00% Test: 76.47%
Run: 07, Epoch: 32, Loss: 0.0779, Train: 99.17%, Valid: 70.00% Test: 76.47%
Run: 07, Epoch: 33, Loss: 0.0814, Train: 100.00%, Valid: 70.00% Test: 76.47%
Run: 07, Epoch: 34, Loss: 0.0763, Train: 100.00%, Valid: 68.75% Test: 78.43%
Run: 07, Epoch: 35, Loss: 0.0596, Train: 100.00%, Valid: 68.75% Test: 78.43%
Run: 07, Epoch: 36, Loss: 0.0388, Train: 100.00%, Valid: 68.75% Test: 78.43%
Run: 07, Epoch: 37, Loss: 0.0748, Train: 100.00%, Valid: 68.75% Test: 78.43%
Run: 07, Epoch: 38, Loss: 0.0613, Train: 100.00%, Valid: 68.75% Test: 78.43%
Run: 07, Epoch: 39, Loss: 0.0488, Train: 100.00%, Valid: 68.75% Test: 78.43%
Run: 07, Epoch: 40, Loss: 0.0781, Train: 100.00%, Valid: 68.75% Test: 78.43%
Run:

Run: 08, Epoch: 61, Loss: 0.0219, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 62, Loss: 0.0335, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 63, Loss: 0.0375, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 64, Loss: 0.0566, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 65, Loss: 0.0179, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 66, Loss: 0.0585, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 67, Loss: 0.0218, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 68, Loss: 0.0348, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 69, Loss: 0.0812, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 70, Loss: 0.0349, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 71, Loss: 0.0348, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 72, Loss: 0.0430, Train: 100.00%, Valid: 82.50% Test: 76.47%
Run: 08, Epoch: 73, Loss: 0.0238, Train: 100.00%, Valid: 82.50% Test: 76.47%

Run: 09, Epoch: 94, Loss: 0.0181, Train: 100.00%, Valid: 68.75% Test: 76.47%
Run: 09, Epoch: 95, Loss: 0.0180, Train: 100.00%, Valid: 68.75% Test: 76.47%
Run: 09, Epoch: 96, Loss: 0.0122, Train: 100.00%, Valid: 68.75% Test: 76.47%
Run: 09, Epoch: 97, Loss: 0.0066, Train: 100.00%, Valid: 68.75% Test: 76.47%
Run: 09, Epoch: 98, Loss: 0.0058, Train: 100.00%, Valid: 70.00% Test: 76.47%
Run: 09, Epoch: 99, Loss: 0.0169, Train: 100.00%, Valid: 68.75% Test: 74.51%
Run: 09, Epoch: 100, Loss: 0.0070, Train: 100.00%, Valid: 68.75% Test: 74.51%
Run 09:
Highest Train: 100.00
Highest Valid: 72.50
  Final Train: 100.00
   Final Test: 74.51
Run: 10, Epoch: 01, Loss: 1.8049, Train: 66.67%, Valid: 57.50% Test: 52.94%
Run: 10, Epoch: 02, Loss: 0.8613, Train: 78.33%, Valid: 62.50% Test: 60.78%
Run: 10, Epoch: 03, Loss: 0.7309, Train: 84.17%, Valid: 66.25% Test: 64.71%
Run: 10, Epoch: 04, Loss: 0.6518, Train: 85.00%, Valid: 65.00% Test: 72.55%
Run: 10, Epoch: 05, Loss: 0.5694, Train: 85.00%, Valid: 65.00%

# WISE EMBEDDING

In [7]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[251, 1703], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515])


In [8]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,1694,1695,1696,1697,1698,1699,1700,1701,1702,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1


In [9]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [ ]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

In [10]:
Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    #print(f)
    f.clear()
    Fec.append(vec)

In [11]:
print(Fec)

[[46, 72, 68, 59, 56], [39, 47, 49, 45, 45], [42, 48, 51, 49, 48], [27, 32, 33, 29, 27], [36, 64, 63, 58, 43], [81, 145, 129, 122, 103], [131, 242, 208, 176, 165], [35, 48, 48, 45, 37], [52, 69, 78, 67, 61], [85, 82, 83, 78, 72], [41, 43, 47, 45, 42], [52, 65, 70, 67, 60], [45, 53, 60, 55, 50], [50, 62, 74, 62, 58], [37, 42, 42, 42, 38], [50, 53, 63, 60, 56], [45, 47, 54, 45, 43], [36, 39, 41, 37, 34], [47, 61, 66, 66, 52], [22, 26, 26, 24, 22], [30, 48, 46, 39, 38], [79, 129, 115, 92, 88], [26, 30, 33, 28, 25], [44, 56, 67, 60, 56], [61, 97, 88, 76, 64], [82, 113, 116, 126, 99], [55, 94, 86, 83, 66], [81, 120, 126, 141, 120], [76, 142, 122, 112, 100], [46, 56, 69, 63, 59], [14, 14, 14, 14, 14], [75, 92, 108, 105, 92], [65, 72, 74, 65, 62], [28, 34, 36, 33, 28], [113, 205, 180, 177, 159], [138, 249, 220, 179, 168], [36, 74, 70, 68, 60], [83, 100, 114, 118, 98], [90, 152, 126, 112, 108], [36, 39, 41, 41, 38], [42, 49, 59, 44, 46], [55, 60, 67, 61, 57], [47, 84, 72, 69, 60], [42, 54, 70,

In [12]:
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [13]:
print(SFec)

[[46, 63, 40, 52, 44], [39, 38, 40, 44, 38], [42, 34, 40, 43, 42], [27, 27, 27, 26, 24], [36, 54, 36, 43, 35], [81, 105, 54, 81, 78], [131, 174, 63, 104, 115], [35, 35, 27, 36, 32], [52, 49, 48, 54, 49], [85, 55, 43, 60, 59], [41, 35, 39, 41, 39], [52, 42, 47, 59, 52], [45, 32, 39, 46, 41], [50, 43, 41, 55, 45], [37, 33, 37, 39, 35], [50, 40, 47, 56, 46], [45, 35, 37, 32, 37], [36, 30, 36, 32, 29], [47, 42, 41, 52, 43], [22, 21, 22, 21, 22], [30, 38, 25, 32, 30], [79, 103, 44, 66, 62], [26, 25, 28, 23, 22], [44, 37, 37, 56, 49], [61, 86, 38, 52, 45], [82, 68, 50, 97, 75], [55, 78, 39, 56, 53], [81, 53, 47, 118, 95], [76, 113, 45, 76, 70], [46, 40, 47, 61, 48], [14, 14, 14, 14, 14], [75, 56, 58, 93, 78], [65, 49, 49, 52, 51], [28, 29, 31, 28, 27], [113, 121, 58, 126, 123], [138, 182, 64, 105, 118], [36, 55, 31, 47, 44], [83, 56, 56, 105, 85], [90, 116, 43, 76, 82], [36, 34, 34, 37, 32], [42, 34, 36, 31, 35], [55, 43, 53, 55, 50], [47, 69, 40, 52, 47], [42, 32, 32, 42, 41], [75, 60, 45, 

In [14]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[46., 72., 68.,  ..., 40., 52., 44.],
        [39., 47., 49.,  ..., 40., 44., 38.],
        [42., 48., 51.,  ..., 40., 43., 42.],
        ...,
        [22., 29., 34.,  ..., 22., 26., 27.],
        [53., 86., 77.,  ..., 37., 49., 44.],
        [50., 72., 86.,  ..., 35., 39., 45.]])


'torch.FloatTensor'

# W-SAGE

In [15]:
data.x=CC_domain
print(data)

Data(x=[251, 10], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515])


In [17]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.9758, Train: 31.67%, Valid: 25.00% Test: 23.53%
Run: 01, Epoch: 02, Loss: 1.6533, Train: 34.17%, Valid: 25.00% Test: 21.57%
Run: 01, Epoch: 03, Loss: 1.4110, Train: 41.67%, Valid: 42.50% Test: 35.29%
Run: 01, Epoch: 04, Loss: 1.2888, Train: 51.67%, Valid: 57.50% Test: 62.75%
Run: 01, Epoch: 05, Loss: 1.2015, Train: 48.33%, Valid: 55.00% Test: 62.75%
Run: 01, Epoch: 06, Loss: 1.1371, Train: 49.17%, Valid: 53.75% Test: 60.78%
Run: 01, Epoch: 07, Loss: 1.0975, Train: 50.83%, Valid: 53.75% Test: 58.82%
Run: 01, Epoch: 08, Loss: 1.0935, Train: 51.67%, Valid: 55.00% Test: 58.82%
Run: 01, Epoch: 09, Loss: 1.0101, Train: 58.33%, Valid: 57.50% Test: 64.71%
Run: 01, Epoch: 10, Loss: 0.9803, Train: 65.83%, Valid: 66.25% Test: 68.63%
Run: 01, Epoch: 11, Loss: 0.9253, Train: 75.83%, Valid: 70.00% Test: 72.55%
Run: 01, Epoch: 12, Loss: 0.8705, Train: 77.50%, Valid: 75.00% Test: 76.47%
Run: 01, Epoch: 13, Loss: 0.8772, Train: 69.17%, Valid: 72.50% Test: 68.63%
Run: 01, Epo

Run: 02, Epoch: 67, Loss: 0.2314, Train: 94.17%, Valid: 78.75% Test: 94.12%
Run: 02, Epoch: 68, Loss: 0.2890, Train: 92.50%, Valid: 77.50% Test: 94.12%
Run: 02, Epoch: 69, Loss: 0.1805, Train: 93.33%, Valid: 77.50% Test: 94.12%
Run: 02, Epoch: 70, Loss: 0.2059, Train: 90.83%, Valid: 80.00% Test: 92.16%
Run: 02, Epoch: 71, Loss: 0.2365, Train: 90.83%, Valid: 81.25% Test: 94.12%
Run: 02, Epoch: 72, Loss: 0.2317, Train: 91.67%, Valid: 81.25% Test: 94.12%
Run: 02, Epoch: 73, Loss: 0.2489, Train: 90.83%, Valid: 83.75% Test: 94.12%
Run: 02, Epoch: 74, Loss: 0.1964, Train: 90.00%, Valid: 82.50% Test: 92.16%
Run: 02, Epoch: 75, Loss: 0.1794, Train: 90.83%, Valid: 82.50% Test: 92.16%
Run: 02, Epoch: 76, Loss: 0.2495, Train: 90.83%, Valid: 81.25% Test: 92.16%
Run: 02, Epoch: 77, Loss: 0.2050, Train: 94.17%, Valid: 82.50% Test: 94.12%
Run: 02, Epoch: 78, Loss: 0.2002, Train: 94.17%, Valid: 82.50% Test: 94.12%
Run: 02, Epoch: 79, Loss: 0.1796, Train: 95.00%, Valid: 82.50% Test: 94.12%
Run: 02, Epo

Run: 04, Epoch: 60, Loss: 0.3346, Train: 88.33%, Valid: 88.75% Test: 78.43%
Run: 04, Epoch: 61, Loss: 0.3346, Train: 88.33%, Valid: 88.75% Test: 82.35%
Run: 04, Epoch: 62, Loss: 0.3002, Train: 89.17%, Valid: 87.50% Test: 76.47%
Run: 04, Epoch: 63, Loss: 0.3272, Train: 90.83%, Valid: 86.25% Test: 80.39%
Run: 04, Epoch: 64, Loss: 0.3434, Train: 90.83%, Valid: 85.00% Test: 86.27%
Run: 04, Epoch: 65, Loss: 0.2987, Train: 92.50%, Valid: 86.25% Test: 88.24%
Run: 04, Epoch: 66, Loss: 0.3189, Train: 92.50%, Valid: 86.25% Test: 88.24%
Run: 04, Epoch: 67, Loss: 0.3832, Train: 91.67%, Valid: 86.25% Test: 86.27%
Run: 04, Epoch: 68, Loss: 0.3003, Train: 91.67%, Valid: 85.00% Test: 86.27%
Run: 04, Epoch: 69, Loss: 0.3228, Train: 94.17%, Valid: 87.50% Test: 88.24%
Run: 04, Epoch: 70, Loss: 0.2713, Train: 94.17%, Valid: 90.00% Test: 90.20%
Run: 04, Epoch: 71, Loss: 0.3181, Train: 92.50%, Valid: 87.50% Test: 84.31%
Run: 04, Epoch: 72, Loss: 0.2661, Train: 91.67%, Valid: 86.25% Test: 74.51%
Run: 04, Epo

Run: 06, Epoch: 53, Loss: 0.3297, Train: 91.67%, Valid: 85.00% Test: 76.47%
Run: 06, Epoch: 54, Loss: 0.4072, Train: 92.50%, Valid: 85.00% Test: 78.43%
Run: 06, Epoch: 55, Loss: 0.2803, Train: 92.50%, Valid: 86.25% Test: 78.43%
Run: 06, Epoch: 56, Loss: 0.2691, Train: 94.17%, Valid: 87.50% Test: 78.43%
Run: 06, Epoch: 57, Loss: 0.2531, Train: 93.33%, Valid: 87.50% Test: 80.39%
Run: 06, Epoch: 58, Loss: 0.3567, Train: 90.00%, Valid: 87.50% Test: 78.43%
Run: 06, Epoch: 59, Loss: 0.4147, Train: 89.17%, Valid: 87.50% Test: 76.47%
Run: 06, Epoch: 60, Loss: 0.3090, Train: 89.17%, Valid: 87.50% Test: 74.51%
Run: 06, Epoch: 61, Loss: 0.2594, Train: 90.83%, Valid: 87.50% Test: 76.47%
Run: 06, Epoch: 62, Loss: 0.2814, Train: 90.83%, Valid: 88.75% Test: 78.43%
Run: 06, Epoch: 63, Loss: 0.2999, Train: 91.67%, Valid: 90.00% Test: 78.43%
Run: 06, Epoch: 64, Loss: 0.2127, Train: 92.50%, Valid: 91.25% Test: 78.43%
Run: 06, Epoch: 65, Loss: 0.2840, Train: 92.50%, Valid: 92.50% Test: 78.43%
Run: 06, Epo

Run: 08, Epoch: 47, Loss: 0.4655, Train: 84.17%, Valid: 86.25% Test: 80.39%
Run: 08, Epoch: 48, Loss: 0.4149, Train: 83.33%, Valid: 85.00% Test: 80.39%
Run: 08, Epoch: 49, Loss: 0.4410, Train: 85.00%, Valid: 82.50% Test: 82.35%
Run: 08, Epoch: 50, Loss: 0.4148, Train: 86.67%, Valid: 86.25% Test: 82.35%
Run: 08, Epoch: 51, Loss: 0.3686, Train: 86.67%, Valid: 87.50% Test: 86.27%
Run: 08, Epoch: 52, Loss: 0.3979, Train: 86.67%, Valid: 87.50% Test: 86.27%
Run: 08, Epoch: 53, Loss: 0.4111, Train: 86.67%, Valid: 88.75% Test: 88.24%
Run: 08, Epoch: 54, Loss: 0.3829, Train: 87.50%, Valid: 88.75% Test: 88.24%
Run: 08, Epoch: 55, Loss: 0.3689, Train: 87.50%, Valid: 90.00% Test: 88.24%
Run: 08, Epoch: 56, Loss: 0.3461, Train: 87.50%, Valid: 88.75% Test: 88.24%
Run: 08, Epoch: 57, Loss: 0.3439, Train: 88.33%, Valid: 85.00% Test: 90.20%
Run: 08, Epoch: 58, Loss: 0.3954, Train: 86.67%, Valid: 87.50% Test: 90.20%
Run: 08, Epoch: 59, Loss: 0.3751, Train: 88.33%, Valid: 90.00% Test: 92.16%
Run: 08, Epo

Run: 10, Epoch: 45, Loss: 0.3604, Train: 88.33%, Valid: 78.75% Test: 78.43%
Run: 10, Epoch: 46, Loss: 0.3245, Train: 90.00%, Valid: 80.00% Test: 80.39%
Run: 10, Epoch: 47, Loss: 0.3024, Train: 88.33%, Valid: 80.00% Test: 82.35%
Run: 10, Epoch: 48, Loss: 0.3006, Train: 85.00%, Valid: 78.75% Test: 82.35%
Run: 10, Epoch: 49, Loss: 0.3535, Train: 84.17%, Valid: 77.50% Test: 82.35%
Run: 10, Epoch: 50, Loss: 0.3283, Train: 84.17%, Valid: 77.50% Test: 82.35%
Run: 10, Epoch: 51, Loss: 0.2838, Train: 85.00%, Valid: 77.50% Test: 82.35%
Run: 10, Epoch: 52, Loss: 0.3810, Train: 86.67%, Valid: 78.75% Test: 82.35%
Run: 10, Epoch: 53, Loss: 0.3080, Train: 84.17%, Valid: 77.50% Test: 82.35%
Run: 10, Epoch: 54, Loss: 0.2941, Train: 82.50%, Valid: 76.25% Test: 82.35%
Run: 10, Epoch: 55, Loss: 0.2565, Train: 86.67%, Valid: 78.75% Test: 84.31%
Run: 10, Epoch: 56, Loss: 0.2932, Train: 92.50%, Valid: 82.50% Test: 86.27%
Run: 10, Epoch: 57, Loss: 0.2767, Train: 93.33%, Valid: 83.75% Test: 88.24%
Run: 10, Epo

# Topological encodding 

In [4]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin')
data = dataset[0]
print(data)

Data(x=[251, 1703], edge_index=[2, 515], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10])


In [5]:
print(data.edge_index.numpy())

[[  0   0   0 ... 248 249 250]
 [ 20  28  99 ... 132 117  35]]


In [8]:
Number_nodes=len(data.y)
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [9]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [10]:
print(G.number_of_edges())

515


In [11]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [12]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [13]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

1  |  59 

2  |  53 

3  |  47 

4  |  27 

5  |  21 

6  |  17 

8  |  4 

9  |  5 

10  |  4 

11  |  5 

12  |  3 

13  |  2 

14  |  1 

18  |  1 

21  |  1 

122  |  1 



In [14]:
import pyflagser
Node_fil=[1,2,3,4,5,6,7,8,9,10,20,50]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 250 (100%)

In [15]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[251, 1703], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515])


In [16]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

In [30]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[251, 10], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515], topo=[251, 24])


# TOPO-W-GSAGE

In [17]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x
        #return x.log_softmax(dim=-1)

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [34]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.5083, Train: 14.17%, Valid: 8.75% Test: 15.69%
Run: 01, Epoch: 02, Loss: 1.4521, Train: 14.17%, Valid: 8.75% Test: 15.69%
Run: 01, Epoch: 03, Loss: 1.4165, Train: 54.17%, Valid: 58.75% Test: 68.63%
Run: 01, Epoch: 04, Loss: 1.3621, Train: 52.50%, Valid: 63.75% Test: 60.78%
Run: 01, Epoch: 05, Loss: 1.3217, Train: 52.50%, Valid: 61.25% Test: 58.82%
Run: 01, Epoch: 06, Loss: 1.2817, Train: 53.33%, Valid: 63.75% Test: 60.78%
Run: 01, Epoch: 07, Loss: 1.2651, Train: 55.00%, Valid: 65.00% Test: 62.75%
Run: 01, Epoch: 08, Loss: 1.2042, Train: 57.50%, Valid: 63.75% Test: 62.75%
Run: 01, Epoch: 09, Loss: 1.1916, Train: 58.33%, Valid: 63.75% Test: 64.71%
Run: 01, Epoch: 10, Loss: 1.1729, Train: 57.50%, Valid: 62.50% Test: 64.71%
Run: 01, Epoch: 11, Loss: 1.1709, Train: 60.83%, Valid: 63.75% Test: 64.71%
Run: 01, Epoch: 12, Loss: 1.1086, Train: 61.67%, Valid: 63.75% Test: 64.71%
Run: 01, Epoch: 13, Loss: 1.0786, Train: 60.00%, Valid: 65.00% Test: 66.67%
Run: 01, Epoch

Run: 01, Epoch: 114, Loss: 0.4015, Train: 86.67%, Valid: 80.00% Test: 78.43%
Run: 01, Epoch: 115, Loss: 0.4004, Train: 86.67%, Valid: 80.00% Test: 78.43%
Run: 01, Epoch: 116, Loss: 0.3957, Train: 86.67%, Valid: 80.00% Test: 78.43%
Run: 01, Epoch: 117, Loss: 0.3633, Train: 90.83%, Valid: 81.25% Test: 80.39%
Run: 01, Epoch: 118, Loss: 0.4275, Train: 93.33%, Valid: 81.25% Test: 80.39%
Run: 01, Epoch: 119, Loss: 0.3990, Train: 95.83%, Valid: 80.00% Test: 82.35%
Run: 01, Epoch: 120, Loss: 0.3879, Train: 95.83%, Valid: 80.00% Test: 84.31%
Run: 01, Epoch: 121, Loss: 0.3687, Train: 94.17%, Valid: 80.00% Test: 80.39%
Run: 01, Epoch: 122, Loss: 0.3509, Train: 95.00%, Valid: 83.75% Test: 80.39%
Run: 01, Epoch: 123, Loss: 0.3334, Train: 93.33%, Valid: 82.50% Test: 78.43%
Run: 01, Epoch: 124, Loss: 0.4049, Train: 96.67%, Valid: 85.00% Test: 80.39%
Run: 01, Epoch: 125, Loss: 0.3412, Train: 96.67%, Valid: 85.00% Test: 84.31%
Run: 01, Epoch: 126, Loss: 0.3734, Train: 93.33%, Valid: 82.50% Test: 88.24%

Run: 02, Epoch: 33, Loss: 0.7790, Train: 75.00%, Valid: 65.00% Test: 86.27%
Run: 02, Epoch: 34, Loss: 0.7297, Train: 76.67%, Valid: 66.25% Test: 86.27%
Run: 02, Epoch: 35, Loss: 0.7167, Train: 77.50%, Valid: 66.25% Test: 86.27%
Run: 02, Epoch: 36, Loss: 0.7213, Train: 77.50%, Valid: 66.25% Test: 86.27%
Run: 02, Epoch: 37, Loss: 0.7491, Train: 80.00%, Valid: 66.25% Test: 88.24%
Run: 02, Epoch: 38, Loss: 0.7140, Train: 80.00%, Valid: 67.50% Test: 88.24%
Run: 02, Epoch: 39, Loss: 0.6872, Train: 81.67%, Valid: 68.75% Test: 88.24%
Run: 02, Epoch: 40, Loss: 0.6900, Train: 82.50%, Valid: 70.00% Test: 88.24%
Run: 02, Epoch: 41, Loss: 0.6608, Train: 84.17%, Valid: 71.25% Test: 88.24%
Run: 02, Epoch: 42, Loss: 0.6943, Train: 83.33%, Valid: 71.25% Test: 88.24%
Run: 02, Epoch: 43, Loss: 0.6769, Train: 84.17%, Valid: 73.75% Test: 88.24%
Run: 02, Epoch: 44, Loss: 0.6313, Train: 84.17%, Valid: 73.75% Test: 88.24%
Run: 02, Epoch: 45, Loss: 0.6285, Train: 85.00%, Valid: 75.00% Test: 88.24%
Run: 02, Epo

Run: 02, Epoch: 152, Loss: 0.2726, Train: 95.83%, Valid: 81.25% Test: 92.16%
Run: 02, Epoch: 153, Loss: 0.2603, Train: 96.67%, Valid: 82.50% Test: 94.12%
Run: 02, Epoch: 154, Loss: 0.2886, Train: 97.50%, Valid: 78.75% Test: 90.20%
Run: 02, Epoch: 155, Loss: 0.3146, Train: 95.00%, Valid: 75.00% Test: 82.35%
Run: 02, Epoch: 156, Loss: 0.2727, Train: 94.17%, Valid: 71.25% Test: 78.43%
Run: 02, Epoch: 157, Loss: 0.2825, Train: 95.00%, Valid: 70.00% Test: 76.47%
Run: 02, Epoch: 158, Loss: 0.2979, Train: 96.67%, Valid: 73.75% Test: 80.39%
Run: 02, Epoch: 159, Loss: 0.2363, Train: 97.50%, Valid: 77.50% Test: 86.27%
Run: 02, Epoch: 160, Loss: 0.3054, Train: 98.33%, Valid: 85.00% Test: 88.24%
Run: 02, Epoch: 161, Loss: 0.2572, Train: 97.50%, Valid: 83.75% Test: 94.12%
Run: 02, Epoch: 162, Loss: 0.2965, Train: 96.67%, Valid: 82.50% Test: 92.16%
Run: 02, Epoch: 163, Loss: 0.2734, Train: 95.00%, Valid: 81.25% Test: 90.20%
Run: 02, Epoch: 164, Loss: 0.2520, Train: 91.67%, Valid: 78.75% Test: 90.20%

Run: 03, Epoch: 73, Loss: 0.6211, Train: 80.83%, Valid: 80.00% Test: 78.43%
Run: 03, Epoch: 74, Loss: 0.5927, Train: 84.17%, Valid: 83.75% Test: 82.35%
Run: 03, Epoch: 75, Loss: 0.5661, Train: 85.83%, Valid: 86.25% Test: 82.35%
Run: 03, Epoch: 76, Loss: 0.5958, Train: 88.33%, Valid: 87.50% Test: 82.35%
Run: 03, Epoch: 77, Loss: 0.5713, Train: 92.50%, Valid: 87.50% Test: 84.31%
Run: 03, Epoch: 78, Loss: 0.5758, Train: 94.17%, Valid: 87.50% Test: 84.31%
Run: 03, Epoch: 79, Loss: 0.5738, Train: 97.50%, Valid: 86.25% Test: 86.27%
Run: 03, Epoch: 80, Loss: 0.5990, Train: 96.67%, Valid: 86.25% Test: 88.24%
Run: 03, Epoch: 81, Loss: 0.5787, Train: 92.50%, Valid: 86.25% Test: 86.27%
Run: 03, Epoch: 82, Loss: 0.5899, Train: 90.00%, Valid: 83.75% Test: 84.31%
Run: 03, Epoch: 83, Loss: 0.5468, Train: 88.33%, Valid: 83.75% Test: 86.27%
Run: 03, Epoch: 84, Loss: 0.5438, Train: 87.50%, Valid: 85.00% Test: 86.27%
Run: 03, Epoch: 85, Loss: 0.5750, Train: 84.17%, Valid: 83.75% Test: 84.31%
Run: 03, Epo

...

Run: 08, Epoch: 20, Loss: 0.9866, Train: 57.50%, Valid: 53.75% Test: 54.90%
Run: 08, Epoch: 21, Loss: 0.9001, Train: 58.33%, Valid: 56.25% Test: 54.90%
Run: 08, Epoch: 22, Loss: 0.9297, Train: 58.33%, Valid: 57.50% Test: 54.90%
Run: 08, Epoch: 23, Loss: 0.9390, Train: 59.17%, Valid: 60.00% Test: 54.90%
Run: 08, Epoch: 24, Loss: 0.8843, Train: 60.00%, Valid: 61.25% Test: 56.86%
Run: 08, Epoch: 25, Loss: 0.8999, Train: 65.83%, Valid: 62.50% Test: 58.82%
Run: 08, Epoch: 26, Loss: 0.8837, Train: 67.50%, Valid: 66.25% Test: 62.75%
Run: 08, Epoch: 27, Loss: 0.8688, Train: 67.50%, Valid: 67.50% Test: 64.71%
Run: 08, Epoch: 28, Loss: 0.8630, Train: 67.50%, Valid: 67.50% Test: 64.71%
Run: 08, Epoch: 29, Loss: 0.8611, Train: 69.17%, Valid: 66.25% Test: 66.67%
Run: 08, Epoch: 30, Loss: 0.8127, Train: 69.17%, Valid: 68.75% Test: 66.67%
Run: 08, Epoch: 31, Loss: 0.8402, Train: 69.17%, Valid: 68.75% Test: 66.67%
Run: 08, Epoch: 32, Loss: 0.8052, Train: 70.00%, Valid: 68.75% Test: 66.67%
Run: 08, Epo

Run: 08, Epoch: 139, Loss: 0.3683, Train: 92.50%, Valid: 88.75% Test: 76.47%
Run: 08, Epoch: 140, Loss: 0.3688, Train: 93.33%, Valid: 92.50% Test: 78.43%
Run: 08, Epoch: 141, Loss: 0.4011, Train: 95.83%, Valid: 92.50% Test: 82.35%
Run: 08, Epoch: 142, Loss: 0.3853, Train: 94.17%, Valid: 91.25% Test: 82.35%
Run: 08, Epoch: 143, Loss: 0.3767, Train: 94.17%, Valid: 88.75% Test: 80.39%
Run: 08, Epoch: 144, Loss: 0.4249, Train: 93.33%, Valid: 87.50% Test: 82.35%
Run: 08, Epoch: 145, Loss: 0.4544, Train: 91.67%, Valid: 87.50% Test: 82.35%
Run: 08, Epoch: 146, Loss: 0.3798, Train: 93.33%, Valid: 87.50% Test: 82.35%
Run: 08, Epoch: 147, Loss: 0.3872, Train: 94.17%, Valid: 90.00% Test: 82.35%
Run: 08, Epoch: 148, Loss: 0.4128, Train: 95.83%, Valid: 92.50% Test: 82.35%
Run: 08, Epoch: 149, Loss: 0.3281, Train: 93.33%, Valid: 91.25% Test: 78.43%
Run: 08, Epoch: 150, Loss: 0.3532, Train: 92.50%, Valid: 90.00% Test: 74.51%
Run: 08, Epoch: 151, Loss: 0.4023, Train: 94.17%, Valid: 90.00% Test: 76.47%

Run: 09, Epoch: 56, Loss: 0.6106, Train: 78.33%, Valid: 73.75% Test: 74.51%
Run: 09, Epoch: 57, Loss: 0.6399, Train: 78.33%, Valid: 75.00% Test: 74.51%
Run: 09, Epoch: 58, Loss: 0.6077, Train: 81.67%, Valid: 76.25% Test: 76.47%
Run: 09, Epoch: 59, Loss: 0.6053, Train: 84.17%, Valid: 80.00% Test: 80.39%
Run: 09, Epoch: 60, Loss: 0.5949, Train: 86.67%, Valid: 82.50% Test: 80.39%
Run: 09, Epoch: 61, Loss: 0.6117, Train: 87.50%, Valid: 85.00% Test: 84.31%
Run: 09, Epoch: 62, Loss: 0.5551, Train: 86.67%, Valid: 85.00% Test: 86.27%
Run: 09, Epoch: 63, Loss: 0.6242, Train: 85.83%, Valid: 82.50% Test: 86.27%
Run: 09, Epoch: 64, Loss: 0.6029, Train: 85.83%, Valid: 80.00% Test: 82.35%
Run: 09, Epoch: 65, Loss: 0.5907, Train: 85.83%, Valid: 78.75% Test: 82.35%
Run: 09, Epoch: 66, Loss: 0.5995, Train: 83.33%, Valid: 77.50% Test: 82.35%
Run: 09, Epoch: 67, Loss: 0.5700, Train: 83.33%, Valid: 77.50% Test: 80.39%
Run: 09, Epoch: 68, Loss: 0.5870, Train: 84.17%, Valid: 80.00% Test: 80.39%
Run: 09, Epo

Run: 09, Epoch: 175, Loss: 0.2952, Train: 97.50%, Valid: 88.75% Test: 90.20%
Run: 09, Epoch: 176, Loss: 0.3113, Train: 97.50%, Valid: 87.50% Test: 88.24%
Run: 09, Epoch: 177, Loss: 0.3130, Train: 95.83%, Valid: 87.50% Test: 88.24%
Run: 09, Epoch: 178, Loss: 0.2794, Train: 95.00%, Valid: 88.75% Test: 88.24%
Run: 09, Epoch: 179, Loss: 0.2902, Train: 96.67%, Valid: 90.00% Test: 90.20%
Run: 09, Epoch: 180, Loss: 0.3380, Train: 97.50%, Valid: 88.75% Test: 92.16%
Run: 09, Epoch: 181, Loss: 0.2690, Train: 96.67%, Valid: 88.75% Test: 90.20%
Run: 09, Epoch: 182, Loss: 0.3327, Train: 97.50%, Valid: 90.00% Test: 90.20%
Run: 09, Epoch: 183, Loss: 0.2911, Train: 95.83%, Valid: 88.75% Test: 90.20%
Run: 09, Epoch: 184, Loss: 0.3421, Train: 95.83%, Valid: 87.50% Test: 88.24%
Run: 09, Epoch: 185, Loss: 0.3090, Train: 95.83%, Valid: 86.25% Test: 88.24%
Run: 09, Epoch: 186, Loss: 0.2569, Train: 96.67%, Valid: 87.50% Test: 88.24%
Run: 09, Epoch: 187, Loss: 0.3099, Train: 98.33%, Valid: 88.75% Test: 92.16%

Run: 10, Epoch: 95, Loss: 0.4697, Train: 90.00%, Valid: 78.75% Test: 80.39%
Run: 10, Epoch: 96, Loss: 0.4381, Train: 89.17%, Valid: 80.00% Test: 80.39%
Run: 10, Epoch: 97, Loss: 0.4799, Train: 87.50%, Valid: 82.50% Test: 80.39%
Run: 10, Epoch: 98, Loss: 0.4644, Train: 87.50%, Valid: 82.50% Test: 80.39%
Run: 10, Epoch: 99, Loss: 0.4339, Train: 89.17%, Valid: 82.50% Test: 80.39%
Run: 10, Epoch: 100, Loss: 0.4201, Train: 90.00%, Valid: 82.50% Test: 84.31%
Run: 10, Epoch: 101, Loss: 0.5098, Train: 91.67%, Valid: 81.25% Test: 84.31%
Run: 10, Epoch: 102, Loss: 0.4313, Train: 91.67%, Valid: 83.75% Test: 84.31%
Run: 10, Epoch: 103, Loss: 0.4588, Train: 90.83%, Valid: 83.75% Test: 84.31%
Run: 10, Epoch: 104, Loss: 0.4757, Train: 91.67%, Valid: 81.25% Test: 82.35%
Run: 10, Epoch: 105, Loss: 0.4233, Train: 90.83%, Valid: 82.50% Test: 82.35%
Run: 10, Epoch: 106, Loss: 0.4494, Train: 90.00%, Valid: 82.50% Test: 80.39%
Run: 10, Epoch: 107, Loss: 0.4492, Train: 91.67%, Valid: 82.50% Test: 80.39%
Run:

# TOPO-GSAGE

In [18]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
data = dataset[0]
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[251, 1703], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515], topo=[251, 24])


In [19]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8350, Train: 3.33%, Valid: 6.25% Test: 1.96%
Run: 01, Epoch: 02, Loss: 1.6979, Train: 3.33%, Valid: 6.25% Test: 1.96%
Run: 01, Epoch: 03, Loss: 1.6062, Train: 3.33%, Valid: 6.25% Test: 1.96%
Run: 01, Epoch: 04, Loss: 1.5003, Train: 3.33%, Valid: 6.25% Test: 1.96%
Run: 01, Epoch: 05, Loss: 1.4125, Train: 3.33%, Valid: 6.25% Test: 1.96%
Run: 01, Epoch: 06, Loss: 1.3367, Train: 22.50%, Valid: 21.25% Test: 7.84%
Run: 01, Epoch: 07, Loss: 1.2532, Train: 41.67%, Valid: 52.50% Test: 49.02%
Run: 01, Epoch: 08, Loss: 1.1519, Train: 45.00%, Valid: 52.50% Test: 52.94%
Run: 01, Epoch: 09, Loss: 1.1172, Train: 51.67%, Valid: 52.50% Test: 52.94%
Run: 01, Epoch: 10, Loss: 1.0528, Train: 60.00%, Valid: 53.75% Test: 52.94%
Run: 01, Epoch: 11, Loss: 1.0066, Train: 64.17%, Valid: 55.00% Test: 54.90%
Run: 01, Epoch: 12, Loss: 0.9572, Train: 65.83%, Valid: 58.75% Test: 54.90%
Run: 01, Epoch: 13, Loss: 0.9347, Train: 67.50%, Valid: 61.25% Test: 56.86%
Run: 01, Epoch: 14, Loss: 0.

/Users/joshem/anaconda3/envs/tensorflow/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Run: 01, Epoch: 16, Loss: 0.8280, Train: 70.00%, Valid: 66.25% Test: 60.78%
Run: 01, Epoch: 17, Loss: 0.7981, Train: 74.17%, Valid: 66.25% Test: 62.75%
Run: 01, Epoch: 18, Loss: 0.7872, Train: 75.83%, Valid: 67.50% Test: 62.75%
Run: 01, Epoch: 19, Loss: 0.7605, Train: 76.67%, Valid: 67.50% Test: 62.75%
Run: 01, Epoch: 20, Loss: 0.7559, Train: 79.17%, Valid: 68.75% Test: 62.75%
Run: 01, Epoch: 21, Loss: 0.7140, Train: 80.83%, Valid: 68.75% Test: 66.67%
Run: 01, Epoch: 22, Loss: 0.7089, Train: 82.50%, Valid: 68.75% Test: 66.67%
Run: 01, Epoch: 23, Loss: 0.7126, Train: 86.67%, Valid: 70.00% Test: 66.67%
Run: 01, Epoch: 24, Loss: 0.6754, Train: 86.67%, Valid: 70.00% Test: 66.67%
Run: 01, Epoch: 25, Loss: 0.6542, Train: 86.67%, Valid: 70.00% Test: 66.67%
Run: 01, Epoch: 26, Loss: 0.6593, Train: 87.50%, Valid: 71.25% Test: 66.67%
Run: 01, Epoch: 27, Loss: 0.6295, Train: 87.50%, Valid: 71.25% Test: 66.67%
Run: 01, Epoch: 28, Loss: 0.6508, Train: 87.50%, Valid: 71.25% Test: 66.67%
Run: 01, Epo

Run: 01, Epoch: 147, Loss: 0.2318, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 148, Loss: 0.2701, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 149, Loss: 0.2366, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 150, Loss: 0.2201, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 151, Loss: 0.2295, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 152, Loss: 0.2245, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 153, Loss: 0.2096, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 154, Loss: 0.2402, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 155, Loss: 0.2192, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 156, Loss: 0.1980, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 157, Loss: 0.2076, Train: 96.67%, Valid: 76.25% Test: 70.59%
Run: 01, Epoch: 158, Loss: 0.2155, Train: 96.67%, Valid: 76.25% Test: 72.55%
Run: 01, Epoch: 159, Loss: 0.2165, Train: 96.67%, Valid: 76.25% Test: 72.55%

Run: 02, Epoch: 54, Loss: 0.3762, Train: 98.33%, Valid: 71.25% Test: 78.43%
Run: 02, Epoch: 55, Loss: 0.3713, Train: 98.33%, Valid: 71.25% Test: 78.43%
Run: 02, Epoch: 56, Loss: 0.3937, Train: 99.17%, Valid: 71.25% Test: 78.43%
Run: 02, Epoch: 57, Loss: 0.4001, Train: 99.17%, Valid: 71.25% Test: 78.43%
Run: 02, Epoch: 58, Loss: 0.3757, Train: 99.17%, Valid: 71.25% Test: 78.43%
Run: 02, Epoch: 59, Loss: 0.3729, Train: 99.17%, Valid: 71.25% Test: 78.43%
Run: 02, Epoch: 60, Loss: 0.3881, Train: 99.17%, Valid: 72.50% Test: 78.43%
Run: 02, Epoch: 61, Loss: 0.3683, Train: 99.17%, Valid: 72.50% Test: 78.43%
Run: 02, Epoch: 62, Loss: 0.3456, Train: 99.17%, Valid: 72.50% Test: 78.43%
Run: 02, Epoch: 63, Loss: 0.3627, Train: 99.17%, Valid: 72.50% Test: 78.43%
Run: 02, Epoch: 64, Loss: 0.3619, Train: 99.17%, Valid: 72.50% Test: 78.43%
Run: 02, Epoch: 65, Loss: 0.3397, Train: 99.17%, Valid: 72.50% Test: 78.43%
Run: 02, Epoch: 66, Loss: 0.3232, Train: 99.17%, Valid: 71.25% Test: 78.43%
Run: 02, Epo

...

Run: 08, Epoch: 63, Loss: 0.4902, Train: 94.17%, Valid: 73.75% Test: 66.67%
Run: 08, Epoch: 64, Loss: 0.4874, Train: 95.00%, Valid: 73.75% Test: 66.67%
Run: 08, Epoch: 65, Loss: 0.4356, Train: 95.00%, Valid: 73.75% Test: 66.67%
Run: 08, Epoch: 66, Loss: 0.4544, Train: 95.83%, Valid: 72.50% Test: 64.71%
Run: 08, Epoch: 67, Loss: 0.4265, Train: 95.83%, Valid: 72.50% Test: 68.63%
Run: 08, Epoch: 68, Loss: 0.4046, Train: 96.67%, Valid: 72.50% Test: 70.59%
Run: 08, Epoch: 69, Loss: 0.3926, Train: 97.50%, Valid: 72.50% Test: 70.59%
Run: 08, Epoch: 70, Loss: 0.4075, Train: 97.50%, Valid: 72.50% Test: 68.63%
Run: 08, Epoch: 71, Loss: 0.3768, Train: 97.50%, Valid: 73.75% Test: 68.63%
Run: 08, Epoch: 72, Loss: 0.4059, Train: 97.50%, Valid: 73.75% Test: 68.63%
Run: 08, Epoch: 73, Loss: 0.4296, Train: 97.50%, Valid: 75.00% Test: 66.67%
Run: 08, Epoch: 74, Loss: 0.4199, Train: 97.50%, Valid: 75.00% Test: 68.63%
Run: 08, Epoch: 75, Loss: 0.3794, Train: 97.50%, Valid: 75.00% Test: 68.63%
Run: 08, Epo

Run: 08, Epoch: 183, Loss: 0.1704, Train: 100.00%, Valid: 65.00% Test: 66.67%
Run: 08, Epoch: 184, Loss: 0.1591, Train: 100.00%, Valid: 63.75% Test: 66.67%
Run: 08, Epoch: 185, Loss: 0.1609, Train: 100.00%, Valid: 65.00% Test: 66.67%
Run: 08, Epoch: 186, Loss: 0.2424, Train: 100.00%, Valid: 63.75% Test: 66.67%
Run: 08, Epoch: 187, Loss: 0.1555, Train: 100.00%, Valid: 62.50% Test: 66.67%
Run: 08, Epoch: 188, Loss: 0.1114, Train: 100.00%, Valid: 62.50% Test: 64.71%
Run: 08, Epoch: 189, Loss: 0.1879, Train: 100.00%, Valid: 61.25% Test: 64.71%
Run: 08, Epoch: 190, Loss: 0.1318, Train: 100.00%, Valid: 60.00% Test: 64.71%
Run: 08, Epoch: 191, Loss: 0.1609, Train: 100.00%, Valid: 60.00% Test: 64.71%
Run: 08, Epoch: 192, Loss: 0.1459, Train: 100.00%, Valid: 60.00% Test: 64.71%
Run: 08, Epoch: 193, Loss: 0.1982, Train: 100.00%, Valid: 61.25% Test: 64.71%
Run: 08, Epoch: 194, Loss: 0.1777, Train: 100.00%, Valid: 61.25% Test: 64.71%
Run: 08, Epoch: 195, Loss: 0.1647, Train: 100.00%, Valid: 62.50%

Run: 09, Epoch: 104, Loss: 0.2496, Train: 100.00%, Valid: 70.00% Test: 64.71%
Run: 09, Epoch: 105, Loss: 0.2605, Train: 100.00%, Valid: 68.75% Test: 64.71%
Run: 09, Epoch: 106, Loss: 0.2746, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 09, Epoch: 107, Loss: 0.2663, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 09, Epoch: 108, Loss: 0.2118, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 09, Epoch: 109, Loss: 0.2665, Train: 100.00%, Valid: 68.75% Test: 62.75%
Run: 09, Epoch: 110, Loss: 0.2469, Train: 100.00%, Valid: 67.50% Test: 62.75%
Run: 09, Epoch: 111, Loss: 0.2320, Train: 100.00%, Valid: 67.50% Test: 62.75%
Run: 09, Epoch: 112, Loss: 0.2110, Train: 100.00%, Valid: 67.50% Test: 62.75%
Run: 09, Epoch: 113, Loss: 0.2589, Train: 100.00%, Valid: 67.50% Test: 62.75%
Run: 09, Epoch: 114, Loss: 0.2385, Train: 100.00%, Valid: 67.50% Test: 62.75%
Run: 09, Epoch: 115, Loss: 0.2204, Train: 100.00%, Valid: 67.50% Test: 62.75%
Run: 09, Epoch: 116, Loss: 0.2157, Train: 100.00%, Valid: 67.50%

Run: 10, Epoch: 18, Loss: 0.7391, Train: 76.67%, Valid: 62.50% Test: 58.82%
Run: 10, Epoch: 19, Loss: 0.7578, Train: 77.50%, Valid: 62.50% Test: 62.75%
Run: 10, Epoch: 20, Loss: 0.7111, Train: 77.50%, Valid: 62.50% Test: 64.71%
Run: 10, Epoch: 21, Loss: 0.7314, Train: 77.50%, Valid: 62.50% Test: 66.67%
Run: 10, Epoch: 22, Loss: 0.6836, Train: 77.50%, Valid: 62.50% Test: 66.67%
Run: 10, Epoch: 23, Loss: 0.6792, Train: 77.50%, Valid: 62.50% Test: 66.67%
Run: 10, Epoch: 24, Loss: 0.6843, Train: 77.50%, Valid: 62.50% Test: 66.67%
Run: 10, Epoch: 25, Loss: 0.6687, Train: 78.33%, Valid: 63.75% Test: 64.71%
Run: 10, Epoch: 26, Loss: 0.6653, Train: 79.17%, Valid: 63.75% Test: 64.71%
Run: 10, Epoch: 27, Loss: 0.6401, Train: 81.67%, Valid: 63.75% Test: 64.71%
Run: 10, Epoch: 28, Loss: 0.6287, Train: 83.33%, Valid: 63.75% Test: 64.71%
Run: 10, Epoch: 29, Loss: 0.6150, Train: 84.17%, Valid: 63.75% Test: 64.71%
Run: 10, Epoch: 30, Loss: 0.5975, Train: 85.00%, Valid: 63.75% Test: 64.71%
Run: 10, Epo

Run: 10, Epoch: 146, Loss: 0.2171, Train: 100.00%, Valid: 71.25% Test: 66.67%
Run: 10, Epoch: 147, Loss: 0.2340, Train: 100.00%, Valid: 71.25% Test: 66.67%
Run: 10, Epoch: 148, Loss: 0.2463, Train: 100.00%, Valid: 71.25% Test: 66.67%
Run: 10, Epoch: 149, Loss: 0.2502, Train: 100.00%, Valid: 72.50% Test: 70.59%
Run: 10, Epoch: 150, Loss: 0.1794, Train: 100.00%, Valid: 72.50% Test: 70.59%
Run: 10, Epoch: 151, Loss: 0.2055, Train: 100.00%, Valid: 72.50% Test: 68.63%
Run: 10, Epoch: 152, Loss: 0.2264, Train: 100.00%, Valid: 72.50% Test: 68.63%
Run: 10, Epoch: 153, Loss: 0.1889, Train: 100.00%, Valid: 72.50% Test: 68.63%
Run: 10, Epoch: 154, Loss: 0.2097, Train: 100.00%, Valid: 73.75% Test: 68.63%
Run: 10, Epoch: 155, Loss: 0.2120, Train: 100.00%, Valid: 73.75% Test: 68.63%
Run: 10, Epoch: 156, Loss: 0.1844, Train: 100.00%, Valid: 73.75% Test: 66.67%
Run: 10, Epoch: 157, Loss: 0.2111, Train: 100.00%, Valid: 73.75% Test: 66.67%
Run: 10, Epoch: 158, Loss: 0.2095, Train: 100.00%, Valid: 72.50%